## LangChain 라이브러리를 활용해보자

### LangChain(2022.10)
- 자연어처리(NLP)와 사전 학습된 LLM모델을 사용하여 다양한 응용프로그램을 구축할 수 있게 해주는 라이브럴(설계 및 배포 기능까지 지원함)
- 데이터베이스, API, 외부서비스 등과 통합을 지원하는 인터페이스 제공(openai외에도 Antropic, Hugging Face, Google등 다양한 기업들의 LLM과 연동됨)
- RAG 구현을 위한 기본적인 인터페이스와 임베딩, Advanced RAG 등의 기능을 제공
- ALMessage, UserMessage, SystemMessage 등의 클래스로 대화 메세지를 처리하고 관리함
- 공식 페이지 : http//www.langchain.com/
- langchain docs 페이지 : http://python.langchain.com/v0.2/docs/introduction/

In [ ]:
# !pip install langchain

In [ ]:
# # 랭체인과 오픈ai의 gpt 결합 기능 제공
# !pip install langchain-openai

In [ ]:
# # Langchain 생태계를 확장하고 기본 기능을 보완하는 다양한 도구의 통합을 제공
# !pip install langchain_community

### 모델 로드 및 질의

In [1]:
from langchain_openai import ChatOpenAI
from getpass import getpass

In [15]:
MY_API_KEY = getpass.getpass("OpenAI API KEY:")

OpenAI API KEY: ········


In [17]:
chat_model = ChatOpenAI(model="gpt-3.5-turbo",
                       api_key = MY_API_KEY,
                       )
chat_model

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000015FAB5F4340>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000015FAB609DC0>, root_client=<openai.OpenAI object at 0x0000015FAA745550>, root_async_client=<openai.AsyncOpenAI object at 0x0000015FAB5F46A0>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [21]:
type(chat_model)

langchain_openai.chat_models.base.ChatOpenAI

In [23]:
question = "도전적인 스포츠는?"
answer = chat_model.invoke(question) # invoke : Langchain에서 언어 모델의 응답결과를 반환해주는 함수

In [24]:
print(f"질의 : {question}")
print(f"응답 : {answer.content}")

질의 : 도전적인 스포츠는?
응답 : 등산, 서핑, 스카이다이빙, 스노보딩, 스쿼시 같은 스포츠는 도전적이며 스릴 넘치는 활동이다. 또한 막대 헬스, 크로스핏, 무도 등의 운동도 도전적이며 극한의 체력과 인내력을 요구한다.


### 프롬프트 템플릿 활용
- 규격화된 템플릿을 활용하여 좀 더 체계적인 구성으로 질의가 가능
- 모듈화, 코드 재사용성, 일관성(버전관리 및 테스트) 향상

In [26]:
from langchain.prompts import PromptTemplate

In [27]:
# from_template : 문자열 포메팅을 기본적으로 활용하여 템플릿을 설정해주는 함수
pt = PromptTemplate.from_template("{context} 중에서 {topic}은/는?")

# 템플릿 내부에 들어갈 문자열 각각 지정
prompt = pt.format(context="직업",
                   topic = "도전적인 직업"
                  )
 
print(prompt)

직업 중에서 도전적인 직업은/는?


In [28]:
# PromptTemplate을 활용하여 질의에 맞는 응답을 생성하고 출력하는 사용자 정의 함수 선언
def get_answer(context, topic):
    pt = PromptTemplate.from_template("{context} 중에서 {topic}은/는?")
    prompt = pt.format(context=context,
                       topic = topic
                      )

    # 위에서 작성한 템플릿으로 모델에 질의
    answer = chat_model.invoke(prompt)
    
    return {"질의" :prompt, "응답" : answer.content.strip()}
    

In [34]:
print(get_answer('직업', '윤리적인 직업'))

{'질의': '직업 중에서 윤리적인 직업은/는?', '응답': '의사, 간호사, 교사, 법률가, 사회복지사, 언론인, 환경보호단체 활동가와 같은 직업들이 윤리적인 직업으로 평가될 수 있습니다. 이들 직업은 사회적 책임과 윤리적 가치를 중요시하며, 다른 사람들의 안전과 복지를 위해 노력하는 역할을 수행합니다.'}


In [38]:
# 이전 질문을 조금은 기억 하긴 함
question = "방금 너가 응답한 직업의 종류를 모두 알려줘"
answer = chat_model.invoke(question) # invoke : Langchain에서 언어 모델의 응답결과를 반환해주는 함수
print(f"질의 : {question}")
print(f"응답 : {answer.content}")

질의 : 방금 너가 응답한 직업의 종류를 모두 알려줘
응답 : 네가 방금 언급한 직업은 의사와 간호사입니다.
